In [13]:
# retina
%config InlineBackend.figure_format = 'retina'

In [14]:
import os
from pathlib import Path
import pickle
import numpy as np
import gc
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
import eazy

from matplotlib import rcParams
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 25

WD = Path('/data1/hbahk/spherex-photoz/spherex-challenge/challenge')
os.chdir(WD)

from utils import make_eazy_filters_spherex, plot_comp_hexbin

In [15]:
params = {}
params['FILTERS_RES']       = "FILTERS+spherex_reduced_fullband_halfband.res"
params['TEMPLATES_FILE']    = "spherex_brown_cosmos.templates"
params['TEMP_ERR_FILE']     = "../template_error_cosmos2020.txt"
params['TEMP_ERR_A2']       = 0.0 # nullify the template error

params['TEMPLATE_COMBOS']   = 1
params['SYS_ERR']           = 0.01

params['MW_EBV']            = 0.012
params['CAT_HAS_EXTCORR']   = True

params['CATALOG_FILE']      = "data/spherex_synphot_combined.dat" 

dir_output = WD/"output_combined"

if not dir_output.exists():
    dir_output.mkdir()
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params['MAIN_OUTPUT_FILE']  = "output"

params['APPLY_PRIOR']       = True
params['PRIOR_FILE']        = "../prior_K_TAO.dat"
params['PRIOR_FILTER']      = 466  # SPHEREx band corresponding to 2.2um
params['PRIOR_ABZP']        = 23.9
params['PRIOR_FLOOR']       = 0.01

params['FIX_ZSPEC']         = False
params['Z_MIN']             = 0.015
params['Z_MAX']             = 5.8
params['Z_STEP']            = 0.003
params['Z_STEP_TYPE']       = 1

translate_filename = 'dummy.translate'

In [16]:
import gc
gc.collect()

2557

In [ ]:
with open('tempfilt0003_comb.pickle', 'rb') as rfile:
    tempfilt = pickle.load(rfile)

for i in range(17):
    start_id = i * 10000
    end_id = (i+1) * 10000

    ez = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_filename,
                            zeropoint_file=None, params=params, tempfilt=tempfilt, )
    
    if end_id > len(ez.idx):
        end_id = len(ez.idx)
    
    ez.ZML_WITH_PRIOR = True
    ez.fit_catalog(ez.idx[start_id:end_id], n_proc=2, prior=True)
    ez.ZML_WITH_PRIOR = True
    ez.fit_at_zbest(prior=True, nproc=1)
    
    zout, hdu = ez.standard_output(prior=True, beta_prior=False)
    zout[start_id:end_id].write(dir_output/f'output{i:02d}.fits', overwrite=True)
    phdu = fits.PrimaryHDU(data=ez.lnp[start_id:end_id])
    gridhdu = fits.ImageHDU(data=ez.zgrid)
    hdul = fits.HDUList([phdu, gridhdu])
    hdul.writeto(dir_output/f'lnp{i:02d}.fits', overwrite=True)
    del ez, zout, hdu, phdu, gridhdu, hdul
    gc.collect()

In [33]:
params = {}
params['FILTERS_RES']       = "FILTERS+spherex_reduced_fullband_halfband.res"
params['TEMPLATES_FILE']    = "spherex_brown_cosmos.templates"
params['TEMP_ERR_FILE']     = "../template_error_cosmos2020.txt"
params['TEMP_ERR_A2']       = 0.0 # nullify the template error

params['TEMPLATE_COMBOS']   = 1
params['SYS_ERR']           = 0.01

params['MW_EBV']            = 0.012
params['CAT_HAS_EXTCORR']   = True

params['CATALOG_FILE']      = "data/spherex_synphot.dat" 

dir_output = WD/"output"

if not dir_output.exists():
    dir_output.mkdir()
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params['MAIN_OUTPUT_FILE']  = "output"

params['APPLY_PRIOR']       = True
params['PRIOR_FILE']        = "../prior_K_TAO.dat"
params['PRIOR_FILTER']      = 466  # SPHEREx band corresponding to 2.2um
params['PRIOR_ABZP']        = 23.9
params['PRIOR_FLOOR']       = 0.01

params['FIX_ZSPEC']         = False
params['Z_MIN']             = 0.015
params['Z_MAX']             = 5.8
params['Z_STEP']            = 0.003
params['Z_STEP_TYPE']       = 1

translate_filename = 'dummy.translate'

In [ ]:
with open('tempfilt0003.pickle', 'rb') as rfile:
    tempfilt = pickle.load(rfile)

for i in range(17):
    start_id = i * 10000
    end_id = (i+1) * 10000

    ez = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_filename,
                            zeropoint_file=None, params=params, tempfilt=tempfilt, )
    
    if end_id > len(ez.idx):
        end_id = len(ez.idx)
    
    ez.ZML_WITH_PRIOR = True
    ez.fit_catalog(ez.idx[start_id:end_id], n_proc=2, prior=True)
    ez.ZML_WITH_PRIOR = True
    ez.fit_at_zbest(prior=True, nproc=1)
    
    zout, hdu = ez.standard_output(prior=True, beta_prior=False)
    zout[start_id:end_id].write(dir_output/f'output{i:02d}.fits', overwrite=True)
    phdu = fits.PrimaryHDU(data=ez.lnp[start_id:end_id])
    gridhdu = fits.ImageHDU(data=ez.zgrid)
    hdul = fits.HDUList([phdu, gridhdu])
    hdul.writeto(dir_output/f'lnp{i:02d}.fits', overwrite=True)
    del ez, zout, hdu, phdu, gridhdu, hdul
    gc.collect()